# Loading Data into Neo4J

Tutorial Reference : https://www.youtube.com/watch?v=oXziS-PPIUA

### Angad Beer Singh Dhillon, Pulkit Mathur, Prashanth Modak

In [ ]:
import csv
import json
from neo4j.v1 import GraphDatabase,CypherError,basic_auth
import json
import asyncio
import sys

In [ ]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=basic_auth("neo4j", "sqlproject"))

### Warning : A call to this method will drop all data and relations

In [ ]:
def dropData(client):
    with client.session() as session:
        session.run("MATCH (n) DETACH DELETE n")
    print("All Data Successfully Deleted")

In [ ]:
dropData(driver)

### Defining schema constraints

In [ ]:
CATEGORY_CONSTRAINT = '''
CREATE CONSTRAINT ON (c:Category) ASSERT c.name IS UNIQUE;
'''

USER_CONSTRAINT = '''
CREATE CONSTRAINT ON (u:User) ASSERT u.user_id IS UNIQUE;
'''

BUSINESS_CONSTRAINT = '''
CREATE CONSTRAINT ON (b:Business) ASSERT b.business_id IS UNIQUE;
'''

REVIEW_IMPORT_1 = '''
WITH {items} AS reviews
UNWIND reviews AS review
MERGE (b:Business {business_id: review.business_id})
'''

REVIEW_IMPORT_2 = '''
WITH {items} AS reviews
UNWIND reviews AS review
MERGE (u:User {user_id: review.user_id})
'''

REVIEW_IMPORT_3 = '''
WITH {items} AS reviews
UNWIND reviews AS review
MATCH (u:User {user_id: review.user_id})
MATCH (b:Business {business_id: review.business_id})
CREATE (r:Review {review_id: review.review_id})
SET r.text   = review.text,
    r.type   = review.type,
    r.date   = review.date,
    r.cool   = review.cool,
    r.funny  = review.funny,
    r.stars  = review.stars,
    r.useful = review.useful
CREATE (u)-[:WROTE]->(r)
CREATE (r)-[:REVIEWS]->(b)
'''

BUSINESS_IMPORT = '''
WITH {items} AS businesses
UNWIND businesses AS business
CREATE (b:Business {business_id: business.business_id})
SET b.address = business.address,
              b.lat     = business.latitude,
              b.lon     = business.longitude,
              b.name    = business.name,
              b.city    = business.city,
              b.postal_code = business.postal_code,
              b.state = business.state,
              b.review_count = business.review_count,
              b.stars = business.stars,
              b.is_open = CASE WHEN business.open = 1 THEN True ELSE False END,
              b.neighborhood = business.neighborhood
WITH *
UNWIND business.categories AS cat
MERGE (c:Category {name: cat})
MERGE (b)-[:IN_CATEGORY]->(c)
'''

USER_IMPORT = '''
WITH {items} AS users
UNWIND users AS user
CREATE (u:User {user_id: user.user_id})
SET u.name               = user.name,
    u.type               = user.type,
    u.useful             = user.useful,
    u.yelping_since      = user.yelping_since, 
    u.funny              = user.funny,
    u.review_count       = user.review_count,
    u.average_stars      = user.average_stars,
    u.fans               = user.fans,
    u.compliment_cool    = user.compliment_cool,
    u.compliment_cute    = user.compliment_cute,
    u.compliment_funny   = user.compliment_funny,
    u.compliment_hot     = user.compliment_hot,
    u.compliment_list    = user.compliment_list,
    u.compliment_more    = user.compliment_more,
    u.compliment_note    = user.compliment_note,
    u.compliment_photos  = user.compliment_photos,
    u.compliment_plain   = user.compliment_plain,
    u.compliment_profile = user.compliment_profile,
    u.compliment_writer  = user.compliment_writer,
    u.cool               = user.cool
'''

USER_FRIEND_IMPORT = '''
WITH {items} AS users
UNWIND users AS user
MATCH (u:User {user_id: user.user_id})
UNWIND user.friends AS friend
    MATCH (f:User {user_id: friend})
    CREATE (u)-[:FRIENDS]->(f)
'''

TIP_IMPORT = '''
WITH {items} AS tips
UNWIND tips AS tip
MERGE (u:User {user_id: tip.user_id})
MERGE (b:Business {business_id: tip.business_id})
CREATE (u)-[t:TIP]->(b)
SET t.date  = tip.date,
    t.text  = tip.text,
    t.likes = tip.likes,
    t.type  = tip.type
'''

### Data Set files

In [ ]:
REVIEW_FILE = "yelp_academic_dataset_review.json"
TIP_FILE = "yelp_academic_dataset_tip.json"
USER_FILE = "yelp_academic_dataset_user.json"
CHECKIN_FILE = "yelp_academic_dataset_checkin.json"
BUSINESS_FILE = "yelp_academic_dataset_business.json"

files = [REVIEW_FILE, TIP_FILE, USER_FILE, CHECKIN_FILE, BUSINESS_FILE]

In [ ]:
with driver.session() as session:
    session.run("CREATE CONSTRAINT ON (c:Category) ASSERT c.name IS UNIQUE;")

### Method below imports Data using the contraint queries to NEO4J

In [ ]:
def import_all(part):
    print("Current is ",part['datafile'])
    if part['datafile']:
        with open(part['datafile'], "r",encoding="utf-8") as file:
            items = []
            count = 0
            for line in file:
                items.append(json.loads(line))
                count +=1
                if count > part['size']:
                    with driver.session() as session:
                        session.run(part['cypher'], parameters={'items': items})
                    items = []
                    count = 0
    else:
        with driver.session() as session:
            session.run(part['cypher']).consume()

### Defining queries and other constraints

In [ ]:
 queries = [
        {'datafile': USER_FILE, 'cypher': USER_IMPORT, 'size': 40000},
        {'datafile': None, 'cypher': USER_CONSTRAINT, 'size': 1},
        {'datafile': USER_FILE, 'cypher': USER_FRIEND_IMPORT, 'size': 10000},
        {'datafile': None, 'cypher': CATEGORY_CONSTRAINT, 'size': 1},
        {'datafile': BUSINESS_FILE, 'cypher': BUSINESS_IMPORT, 'size': 20000},
        {'datafile': None, 'cypher': BUSINESS_CONSTRAINT, 'size': 1},
        {'datafile': REVIEW_FILE, 'cypher': REVIEW_IMPORT_3, 'size': 20000},
    ]

In [ ]:
for query in queries:
    import_all(query)